In [1]:
import pandas as pd

In [2]:

bankrolls ={}
for i in range(1,100):
    #df =pd.read_csv(f'bayesian/sigmoid_calibration/8Features/predictions/sigmoid_test_data_{i}.csv')
    df =pd.read_csv(f'bayesian/sigmoid_calibration/predictions/test_predictions_{i}.csv')
    df['homediff'] = df['model_home_win_prob'] - df['home_implied_prob']
    df['awaydiff'] = df['model_away_win_prob'] - df['away_implied_prob']
    df.drop_duplicates(subset='id', keep='first', inplace=True)

    df['bethome'] = ((df['homediff'] > 0.01))
    df['betaway'] = ((df['awaydiff'] >  0.01))

    def calculate_bets(df):
        # Initialize bankroll
        bankroll = 2500
        df['game_won'] = 0
        df['game_lost'] = 0
        
        df['win'] =0
        df['loss'] =0
        # New columns for bet_amount, bet_win, and updated bankroll
        df['bet_amount'] = 0
        df['bet_win'] = 0
        df['bankroll'] = 0
        
        win = 0
        loss = 0
        df['win'] = 0
        df['loss'] = 0
        
        
        
    
        for idx, row in df.iterrows():
            bet = 0
            bet_win = 0
            game_won =0
            game_lost =0
            # Calculate bet amount as 5% of current bankroll
            
            
            if idx ==0:
                bet = bankroll * 0.05
                segment_br = bankroll
            elif idx % 5 ==0:
                bet = bankroll * 0.05
                segment_br = bankroll
            else:
                bet = segment_br * 0.05
            

            
            if row['bethome']:
                if row['home_is_winner'] == 1:
                    win_multiplier = (100 / abs(row['home_odds'])) if row['home_odds'] < 0 else (row['home_odds'] / 100)
                    bet_win = bet * win_multiplier
                    bankroll += bet_win  # Add the winnings to the bankroll
                    win+=1
                    game_won =1
                    game_lost =0
                else:
                    bankroll -= bet  # Decrease the bankroll by the bet amount
                    bet_win = 0
                    loss+=1
                    game_won =0
                    game_lost =1
    
            elif row['betaway']:
                if row['home_is_winner'] == 0:
                    win_multiplier = (100 / abs(row['away_odds'])) if row['away_odds'] < 0 else (row['away_odds'] / 100)
                    bet_win = bet * win_multiplier
                    bankroll += bet_win  # Add the winnings to the bankroll
                    win+=1
                    game_won=1
                    game_lost =0
                else:
                    bankroll -= bet  # Decrease the bankroll by the bet amount
                    bet_win = 0
                    loss+=1
                    game_won=0
                    game_lost =1
            else: bet =0 
            # Update the DataFrame
            
            df.at[idx, 'bet_amount'] = bet
            df.at[idx, 'net_bet_win'] = bet_win
            df.at[idx, 'bankroll'] = bankroll
            df.at[idx,'game_won']= game_won
            df.at[idx,'game_lost']= game_lost
            
            df.at[idx, 'total_won_games'] = win
            df.at[idx, 'total_lost_games'] = loss
    
        
        if df['bankroll'].iloc[-1] >1000:
            df.to_csv(f'bayesian/sigmoid_calibration/12features/10pct/win/test_set{i}.csv')
            bankrolls[i] =df['bankroll'].iloc[-1]
        else:
            df.to_csv(f'bayesian/sigmoid_calibration/12features/10pct/lost/test_set{i}.csv')
            bankrolls[i] =df['bankroll'].iloc[-1]
        return df, bankrolls
    df, bankrolls = calculate_bets(df)


In [3]:
df = pd.DataFrame.from_dict(bankrolls,orient='index', columns=['bankroll_totals'])

In [4]:
idx_min = df.idxmin()

In [5]:
df

,bankroll_totals
1,1008.261471
2,5328.243980
3,2261.360461
4,573.515603
5,550.933407
...,...
95,213.220736
96,687.987078
97,321.931241
98,219.695700


In [6]:
idx_min

bankroll_totals    58
dtype: int64

In [7]:
df

,bankroll_totals
1,1008.261471
2,5328.243980
3,2261.360461
4,573.515603
5,550.933407
...,...
95,213.220736
96,687.987078
97,321.931241
98,219.695700


In [8]:
df.to_csv('df_best.csv')

In [11]:
df.describe()

,bankroll_totals
count,99.000000
mean,1796.949439
std,2015.126864
min,62.367222
25%,472.059389
50%,1060.028952
75%,2471.037731
max,12843.824548


In [9]:
df[['bankroll_totals']].median()

bankroll_totals    1060.028952
dtype: float64

In [10]:
df.to_csv('bayesian/sigmoid_calibration/12features/10pct/Bayesian_SigmoidCalibration_10percent.csv')